### Step 1 - Loads packages required

In [1]:
import pandas as pd
import pandasql as ps

### Step 2 - Interface file exploration to validate separators and text qualifiers

In [7]:
from custom_procedures import newestFile
sourceFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/'
sourceFile = newestFile(sourceFolder)
if sourceFile:
    print(sourceFile)
    if os.path.exists(sourceFile):
        #textFile = open(sourceFile, mode="r", encoding="utf-8")
        textFile = open(sourceFile, mode="r")
        if textFile:
            textLine = textFile.readline().strip()
            print(textLine)
            numTextLines = 0
            while textLine:
                #if numTextLines <= 10:
                textLine = textFile.readline().strip()
                #print(textLine)
                numTextLines += 1
            print(numTextLines, ' text lines read')

/content/drive/Shareddrives/Esperanto/Supports/Suppliers/1. SUPPLIERS_INTERFACE_COPPEL.csv
'SISTEMA_LEGADO';'NUMERO_PROVEEDOR';'BATCH_ID';'IMPORT_ACTION';'VENDOR_NAME';'VENDOR_NAME_NEW';'SEGMENT1';'VENDOR_NAME_ALT';'ORGANIZATION_TYPE_LOOKUP_CODE';'VENDOR_TYPE_LOOKUP_CODE';'END_DATE_ACTIVE';'BUSINESS_RELATIONSHIP';'PARENT_SUPPLIER_NAME';'ALIAS';'DUNS_NUMBER';'ONE_TIME_FLAG';'CUSTOMER_NUM';'STANDARD_INDUSTRY_CLASS';'NI_NUMBER';'CORPORATE_WEBSITE';'CHIEF_EXECUTIVE_TITLE';'CHIEF_EXECUTIVE_NAME';'BUS_CLASS_NOT_APPLICABLE';'TAX_COUNTRY_CODE';'NUM_1099';'FEDERAL_REPORTABLE_FLAG';'TYPE_1099';'STATE_REPORTABLE_FLAG';'TAX_REPORTING_NAME';'NAME_CONTROL';'TAX_VERIFICATION_DATE';'ALLOW_AWT_FLAG';'AWT_GROUP_NAME';'VAT_CODE';'VAT_REGISTRATION_NUM';'AUTO_TAX_CALC_OVERRIDE';'PAYMENT_METHOD_LOOKUP_CODE';'DELIVERY_CHANNEL';'BRANK_INSTRUCTION1';'BRANK_INSTRUCTION2';'BRANK_INSTRUCTION_DETAILS';'SETTLEMENT_PRIORITY';'PAYMENT_TEXT_MESSAGE_1';'PAYMENT_TEXT_MESSAGE_2';'PAYMENT_TEXT_MESSAGE_3';'IBY_BANK_CHARGE_

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd1 in position 3774: invalid continuation byte

### Step 3 - Loads interface data

In [ ]:
df = pd.read_csv(sourceFile, sep=';')
if df:
    print(df.columns)

In [ ]:
### Step 4 - 